# <center> Hatree-Fock Method </center>

#### Molecular Schrodinger Equation 
$$H \Psi = E \Psi$$
$$
\left [ 
    - \sum_{i} \frac{\nabla_i^2}{2} 
    - \sum_{i} \frac{\nabla_A^2}{2} 
    - \sum_{A,i} \frac{Z^2}{r_{Ai}}
    + \sum_{A>B} \frac{Z_A Z_B}{R_{AB}}
    + \sum_{i>j} \frac{1}{r_{ij}}
\right ] \Psi(r;R) = E \Psi(r;R)
$$
$$
\left[ \hat{T}_e(r) + \hat{T}_N(r) + \hat{V}_{eN}(r;R) + \hat{V}_{NN}(R) + \hat{V}_{ee}(r) \right] \Psi(r;R) = E \Psi(r;R)
$$
#### After Approximation
$$
\left[ \hat{T}_e(r)  + \hat{V}_{eN}(r;R)  + \hat{V}_{ee}(r) \right] \Psi(r;R) = E \Psi(r;R)
$$

#### Hatree-Fock Equation

$$\left ( \mathcal{H}^{Core} + \sum_i \left [ 2 \mathcal{J}_j - \mathcal{H}_j  \right ] \right ) \psi_i(r_i) = \sum_i \varepsilon_{ij} \psi_i(r_i) $$

$$ f_i \psi_i(r_i) = \sum_i \varepsilon_{ij} \psi_i(r_i) $$

<center> fi is the fock operator </center>

In [40]:
use std::f64::consts::PI;

# Functions

In [41]:
// Linspace 

pub fn linspace(a: f64, b: f64, n: usize) -> Vec<f64> {
    let dx = (b - a) / (n - 1) as f64;
    (0..n).map(|i| a + i as f64 * dx).collect()
}

In [42]:
:dep gnuplot
use gnuplot::*;

pub fn plot2d(x:&Vec<f64>,y:&Vec<f64>) -> Figure{
    let mut fg = Figure::new();
    fg.axes2d()
        .lines(x,y, &[]);
    fg.show().unwrap();
    fg
}

## Slatter Type Orbitals (SLA)

$$ \phi^{SLA} (r) = \left (\frac{\zeta^3}{\pi} \right )^{\frac{1}{2}} e^{-\zeta r}$$

In [43]:
fn psi_sto(zeta: f64, r:&Vec<f64>) -> Vec<f64>{
    let a = (zeta.powi(3) / PI).powf(0.5);
    r.iter().map(|val| a * (- zeta * val).exp()).collect()
}

In [44]:
let mut x:Vec<f64> = linspace(-5.0,5.0,1000);
let mut r:Vec<f64> = x.iter().map(|val| val.abs()).collect();
let mut y = psi_sto(1.0,&r);

let mut psi_sto_plot = plot2d(&x, &y);
// psi_sto_plot.save_to_png("psi_sto.png", 1920,1080)

## Contracted Gaussian Function

$$\phi^{GF}(\alpha) = (2 \alpha / \pi)^{3/4} e^{-\alpha r}$$

$$\phi^{CGF}(r) = \sum_n d_n \phi^{GF}_n(\alpha)$$

$$\phi^{CGF}_{STO-3G}(r) = \sum_n^3 d_n \phi^{GF}_n(\alpha)$$

In [45]:
let Coeff = vec![
    vec![1.00000, 0.0000000, 0.000000],
    vec![0.678914, 0.430129, 0.000000],
    vec![0.444635, 0.535328, 0.154329],
];

let Expon = vec![
    vec![0.270950, 0.000000, 0.000000],
    vec![0.151623, 0.851819, 0.000000],
    vec![0.109818, 0.405771, 2.227660],
];

### psi_CGF_STO1G

In [46]:
let mut psi_CGF_STO1G: Vec<f64> = r.iter()
    .map(|a| (-Expon[0][0] * a.powi(2)).exp() * Coeff[0][0] * (2.0 * Expon[0][0] / PI).powf(0.75) )
    .collect();

let mut fg = Figure::new();
fg.axes2d()
    .lines(&x,&y, &[])
    .lines(&x,&psi_CGF_STO1G,&[]);
fg.show().unwrap();

### psi_CGF_STO2G

In [47]:
let mut psi_CGF_STO2G: Vec<f64> = r.iter()
    .map(
        |a| 
        (-Expon[1][0] * a.powi(2)).exp() * Coeff[1][0] * (2.0 * Expon[1][0] / PI).powf(0.75) 
        + (-Expon[1][1] * a.powi(2)).exp() * Coeff[1][1] * (2.0 * Expon[1][1] / PI).powf(0.75)
        + (-Expon[1][2] * a.powi(2)).exp() * Coeff[1][2] * (2.0 * Expon[1][2] / PI).powf(0.75)
    )
    .collect();

let mut fg = Figure::new();
fg.axes2d()
    .lines(&x,&y, &[])
    .lines(&x,&psi_CGF_STO2G,&[]);
fg.show().unwrap();

### psi_CGF_STO3G

In [48]:
let mut psi_CGF_STO2G: Vec<f64> = r.iter()
    .map(
        |a| 
        (-Expon[2][0] * a.powi(2)).exp() * Coeff[2][0] * (2.0 * Expon[2][0] / PI).powf(0.75) 
        + (-Expon[2][1] * a.powi(2)).exp() * Coeff[2][1] * (2.0 * Expon[2][1] / PI).powf(0.75)
        + (-Expon[2][2] * a.powi(2)).exp() * Coeff[2][2] * (2.0 * Expon[2][2] / PI).powf(0.75)
    )
    .collect();

let mut fg = Figure::new();
fg.axes2d()
    .lines(&x,&y, &[])
    .lines(&x,&psi_CGF_STO2G,&[]);
fg.show().unwrap();

## Integrals

### Overlap Integral

$$S(A,B,Rab2) = \left(\frac{\pi}{A+B}\right)^{3/2} e^{-\dfrac{A\ B \ Rab2}{A+B}}$$

In [49]:
// Calculates the overlap between two gaussian functions
fn Sint(a: f64, b: f64, rab2: f64) -> f64{
    (PI / (a + b)).powf(1.5) * (-a * b * rab2 / (a +b)).exp()
}

In [50]:
// Calculates the kinetic energy integrals for un-normalised primitives
fn Tint(a: f64, b: f64, rab2: f64) -> f64{
    (a * b / (a + b)) * (3.0 - 2.0 * a * b * rab2 / (a + b)) * (PI / (a + b)).powf(1.5) * (-a * b * rab2 / (a +b)).exp()
}

In [51]:
// Mathematical Functions

// F function for 1s orbital
fn F0(t: f64) -> f64 {
    if (t < 1e-6) { 1.0 - t/3.0 }
    else { 0.5 * (PI / t).powf(0.5) * erf(t.powf(0.5)) }
}

// Approximation for the error function
fn erf(t: f64) -> f64 {
    let P = 0.3275911;
    let A = vec![0.254829592,-0.284496736,1.421413741,-1.453152027,1.061405429];
    let T = 1.0/(1.0+P*t);
    let mut Tn = T;
    let mut Poly = A[0] * Tn;
    for i in 1..5{
        Tn = Tn * T;
        Poly = Poly * A[i] * Tn;
    }
    return 1.0 - Poly * (-t.powi(2)).exp();
}

In [52]:
// Calculates the kinetic energy integrals for un-normalised primitives
fn Vint(a: f64, b: f64, rab2: f64, rcp2: f64, zc: f64) -> f64 {
    zc - (2.0 * PI / (a+b) * F0((a+b)*rcp2) * (-a*b*rab2/(a+b)).exp())
}

/*
Calculate two electron integrals A,B,C,D are the exponents alpha, beta, etc.
Rab2 equals squared distance between centre A and centre B
*/
fn TwoE(a: f64, b: f64, c: f64, d: f64, rab2: f64, rcd2: f64, rpq2: f64) -> f64{
    2.0 * PI.powf(2.5) / ((a+b) * (c+d) * (a+b+c+d).sqrt()) * F0((a+b)*(c+d)*rpq2/(a+b+c+d)) 
    * (-a*b*rab2/(a+b) - c*d*rcd2/(c+d)).exp()
}

## Intgrl

In [56]:
fn Intgrl(n: usize, r: f64, zeta1: f64, zeta2: f64, za: f64, zb: f64) -> Vec<f64>{

    let mut r2 = r.powi(2);

    let mut S12 = 0.0;
    let mut T11 = 0.0;
    let mut T12 = 0.0;
    let mut T22 = 0.0;
    let mut V11A = 0.0;
    let mut V12A = 0.0;
    let mut V22A = 0.0;
    let mut V11B = 0.0;
    let mut V12B = 0.0;
    let mut V22B = 0.0;
    let mut V1111 = 0.0;
    let mut V2111 = 0.0;
    let mut V2121 = 0.0;
    let mut V2211 = 0.0;
    let mut V2221 = 0.0;
    let mut V2222 = 0.0;

    let Coeff = vec![
        vec![1.00000, 0.0000000, 0.000000],
        vec![0.678914, 0.430129, 0.000000],
        vec![0.444635, 0.535328, 0.154329],
    ];

    let Expon = vec![
        vec![0.270950, 0.000000, 0.000000],
        vec![0.151623, 0.851819, 0.000000],
        vec![0.109818, 0.405771, 2.227660],
    ];

    let mut D1: Vec<f64> = vec![0.0;3];
    let mut A1: Vec<f64> = vec![0.0;3];
    let mut D2: Vec<f64> = vec![0.0;3];
    let mut A2: Vec<f64> = vec![0.0;3];

    for i in 0..n {
        A1[i] = Expon[n-1][i]*(zeta1.powi(2));
        D1[i] = Coeff[n-1][i]*((2.0*A1[i]/PI).powf(0.75));
        A2[i] = Expon[n-1][i]*(zeta2.powi(2));
        D2[i] = Coeff[n-1][i]*((2.0* A2[i]/PI).powf(0.75));
    }

    
    // Calculate one electron integrals 
    // Centre A is first atom centre B is second atom
    // Origin is on second atom
    // V12A - off diagonal nuclear attraction to centre A etc.
    
    for i in 0..n{
        for j in 0..n{
            let Rap = A2[j] * r / (A1[i] + A2[j]);
            let Rap2 = Rap.powi(2);
            let Rbp2 = (r - Rap).powi(2);
            S12 = S12 + Sint(A1[i], A2[j], r2) * D1[i] * D2[j];
            T11 = T11 + Tint(A1[i], A1[j], 0.0) * D1[i] * D2[j];
            T12 = T12 + Tint(A1[i], A2[j], r2) * D1[i] * D2[j];
            T22 = T22 + Tint(A2[i], A2[j], 0.0) * D1[i] * D2[j];
            V11A = V11A + Vint(A1[i], A2[j], 0.0, 0.0, za) * D1[i] * D2[j];
            V12A = V12A + Vint(A2[i], A2[j], r2, Rap2, za) * D1[i] * D2[j];
            V22A = V22A + Vint(A2[i], A2[j], 0.0, r2, za) * D2[i] * D2[j];
            V11B = V11B + Vint(A1[i], A1[j], 0.0, r2, zb) * D1[i] * D1[j];
            V12B = V12B + Vint(A1[i], A2[j], r2, Rbp2, zb) * D1[i] * D2[j];
            V22B = V22B + Vint(A2[i], A2[j], 0.0, 0.0, zb) * D2[i] * D2[j];
        }
    }

    // Calculate two electron integrals
    for i in 0..n{
        for j in 0..n{
            for k in 0..n{
                for l in 0..n{
                    let Rap = A2[i]*r/(A2[i]+A1[j]);
                    let Rbp = r - Rap;
                    let Raq = A2[k]*r/(A2[k]+A1[l]);
                    let Rbq = r - Raq;
                    let Rpq = Rap - Raq;
                    let Rap2 = Rap*Rap;
                    let Rbp2 = Rbp*Rbp;
                    let Raq2 = Raq*Raq;
                    let Rbq2 = Rbq*Rbq;
                    let Rpq2 = Rpq*Rpq;
                    V1111 = V1111 + TwoE(A1[i],A1[j],A1[k],A1[l],0.0,0.0,0.0) * D1[i] * D1[j] * D1[k] * D1[l];
                    V2111 = V2111 + TwoE(A2[i],A1[j],A1[k],A1[l],r2,0.0,Rap2) * D2[i] * D1[j] * D1[k] * D1[l];
                    V2121 = V2121 + TwoE(A2[i],A1[j],A2[k],A1[l],r2,r2,Rpq2) * D2[i] * D1[j] * D2[k] * D1[l];
                    V2211 = V2211 + TwoE(A2[i],A2[j],A1[k],A1[l],0.0,0.0,r2) * D2[i] * D2[j] * D1[k] * D1[l];
                    V2221 = V2221 + TwoE(A2[i],A2[j],A2[k],A1[l],0.0,r2,Rbq2) * D2[i] * D2[j] * D2[k] * D1[l];
                    V2222 = V2222 + TwoE(A2[i],A2[j],A2[k],A2[l],0.0,0.0,0.0) * D2[i] * D2[j] * D2[k] * D2[l];
                }
            }
        }
    }

    return vec![S12, T11, T12, T22, V11A, V12A, V22A, V11B, V12B, V22B, V1111, V2111, V2121, V2211, V2221, V2222]

}

In [60]:
let n = 3;
let r = 1.4632;
let zeta1 = 1.69;
let zeta2 = 1.24;
let za = 2.0;
let zb = 1.0;
let intgrlvalue = Intgrl(n, r, zeta1, zeta2, za, zb);
let name = vec!["S12", "T11", "T12", "T22", "V11A", "V12A", "V22A", "V11B", "V12B", "V22B", "V1111", "V2111", "V2121", "V2211", "V2221", "V2222"];
for (i,j) in name.iter().zip(intgrlvalue.iter()){
    println!("{} -> {}",i,j);
}



S12 -> 0.5368198483767278
T11 -> 0.8872886514654885
T12 -> 0.19744374308173315
T22 -> 1.2092885653037704
V11A -> -0.14079813805946778
V12A -> -1.6952243676128755
V22A -> 0.10625301368966225
V11B -> 0.31582041506981173
V12B -> -0.1580450711467842
V22B -> -0.8319210511736604
V1111 -> 1.0557162326254552
V2111 -> 1.3989001290257903
V2121 -> 0.5289539126939468
V2211 -> 0.682756538968368
V2221 -> 0.7733247034892219
V2222 -> 0.7746083600328786


()